## parse different journals in xlsx to a single CSV

In [1]:
library(magrittr)
library(readxl)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# load data from all sheet and concat 
sheets <- excel_sheets("./sample data_20200923(5years).xlsx")
sheets <- sheets[-1]
list_all_example <- lapply(sheets, function(x) read_excel("./sample data_20200923(5years).xlsx", sheet = x))

df_doCall <- do.call("rbind", list_all_example)

df <-
  df_doCall %>% 
  select(-c("SerialNumber", "DocType"))

In [3]:
sheets <- excel_sheets("./sample data_20200923(5years).xlsx")
sheets <- sheets[1]
list_all_example <- lapply(sheets, function(x) read_excel("./sample data_20200923(5years).xlsx", sheet = x))
df_doCall <- do.call("rbind", list_all_example)
                           
# Notice that there's no DBYear in the Excel,
# so I impute them manually by looking up journals' website
# and create the file with paper title and year.
# Please ignore the following processes if DBYear is back in the original data.
library(readr)
econometrica_papertitle_filled <- read_csv("econometrica_papertitle_filled.csv")
                           
df_econometrica <-
  df_doCall %>%
  left_join(econometrica_papertitle_filled, by = "PaperTitle") %>%
  select(DBYear = dbYear, everything())


── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  PaperTitle = col_character(),
  dbYear = col_double()
)




In [4]:
df_final <- bind_rows(df, df_econometrica)
rio::export(df_final, "df_final.csv")

## create incidence matrix

In [13]:
library(readr)
library(reshape2)
library(magrittr)
library(dplyr)
library(RSpectra)
library(matlib)

ERROR: Error in library(matlib): there is no package called ‘matlib’


In [2]:
# import data
df <- read_csv("./df_final.csv")

# select researcher, articles and country
df_test <-
  df %>%
  select(Fullname, PaperSecUID, Country) %>%
  # head(100) %>%
  mutate(triangle_id = row_number())

head(df_test)


── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  DBYear = col_double(),
  PaperSecUID = col_double(),
  PaperTitle = col_character(),
  ArticleNumber = col_character(),
  countryNumber = col_double(),
  AuthorNumber = col_double(),
  AffiliationNumber = col_double(),
  Country = col_character(),
  FullOrgName = col_character(),
  OrgFirstClassName = col_character(),
  Fullname = col_character(),
  journaltitle = col_character()
)




Fullname,PaperSecUID,Country,triangle_id
<chr>,<dbl>,<chr>,<int>
"Quintin, Erwan",349557700001,USA,1
"Corbae, Dean",349557700001,USA,2
"Corbae, Dean",349557700001,USA,3
"Zame, William",349557700002,USA,4
"Bossaerts, Peter",349557700002,USA,5
"Bossaerts, Peter",349557700002,Australia,6


In [3]:
res = melt(df_test, id.var = "triangle_id")[-2]
t = table(res)

In [4]:
# create incidence matrix -> article num x len(country + name + article)
incidence_matrix <- table(melt(df_test, id.var = "triangle_id")[-2]) %>% as.matrix()

incidence_matrix_col_names <- table(melt(df_test, id.var = "triangle_id")[-2]) %>% colnames()

# ev <- eigen(t(incidence_matrix)%*%incidence_matrix) # too slow, don't run

print(sprintf("%s: %d", 'nrow', nrow(incidence_matrix)))
print(sprintf("%s: %d", 'ncol', ncol(incidence_matrix)))

[1] "nrow: 6240"
[1] "ncol: 5008"


In [5]:
# calculate vertex ev centrality
start_time <- Sys.time()
res <-
  eigs_sym(t(incidence_matrix) %*% incidence_matrix, 1, which = "LM") # "LM" is the default
end_time <- Sys.time()
end_time - start_time
# Time difference of 2.512093 mins

Time difference of 2.035963 mins

In [6]:
# calculate edge ev centrality
start_time <- Sys.time()
res2 <-
  eigs_sym(incidence_matrix %*% t(incidence_matrix), 1, which = "LM") # "LM" is the default
end_time <- Sys.time()
end_time - start_time
# Time difference of 3.048422 mins

Time difference of 2.457333 mins

In [8]:
# map edge ev centrality back
edge_centrality_df <- data.frame(df_test, edge_centrality = res2$vectors)

# map vertex ev centrality back
vertex_centrality_df <- data.frame(vertex = incidence_matrix_col_names, vertex_centrality = res$vectors)

author_centrality_df <-
  df_test %>%
  select(Fullname) %>%
  distinct(Fullname) %>%
  left_join(vertex_centrality_df, by = c("Fullname" = "vertex"))

paper_centrality_df <-
  df_test %>%
  select(PaperSecUID) %>%
  distinct(PaperSecUID) %>%
  mutate(PaperSecUID = as.character(PaperSecUID)) %>%
  left_join(vertex_centrality_df, by = c("PaperSecUID" = "vertex"))

country_centrality_df <-
  df_test %>%
  select(Country) %>%
  distinct(Country) %>%
  mutate(Country = as.character(Country)) %>%
  left_join(vertex_centrality_df, by = c("Country" = "vertex"))

In [16]:
# top 20 author centrality, all time
author_centrality_df %>%
  arrange(desc(vertex_centrality)) %>%
  head(20)
  # rio::export("top20_author.csv")

# top 20 country centrality, all time
country_centrality_df %>%
  arrange(desc(vertex_centrality)) %>%
  head(20) 
# rio::export("top20_country.csv")

paper_centrality_df %>%
  arrange(desc(vertex_centrality)) %>%
  head(20)

Fullname,vertex_centrality
<chr>,<dbl>
"Akcigit, Ufuk",0.004616596
"Acemoglu, Daron",0.003642605
"Finkelstein, Amy",0.003398558
"Greenstone, Michael",0.003154766
"Pistaferri, Luigi",0.003152534
"Karlan, Dean",0.002911974
"Philippon, Thomas",0.002909621
"Bloom, Nicholas",0.002669560
"Bernard, Andrew B.",0.002668434


Country,vertex_centrality
<chr>,<dbl>
USA,9.991323e-01
England,7.265648e-05
Germany,3.215408e-05
Canada,2.287496e-05
France,1.451913e-05
Spain,1.271163e-05
Italy,1.137855e-05
Switzerland,8.886805e-06
Norway,8.672131e-06


PaperSecUID,vertex_centrality
<chr>,<dbl>
448528900011,0.002916397
498169800005,0.002668239
466609600003,0.002428788
357929400045,0.002426399
371920800003,0.002425568
389459100004,0.002423097
379341300021,0.002182069
380134800004,0.002181952
379341300023,0.002180952


In [12]:
# top 20 author each year, by paper's publish time
author_year_centrality_df <-
  df %>%
  mutate(PaperSecUID = as.character(PaperSecUID)) %>%
  left_join(paper_centrality_df, by = "PaperSecUID") %>%
  group_by(DBYear, Fullname) %>%
  summarise(sum_paper_centrality = sum(vertex_centrality)) %>%
  ungroup()

author_year_centrality_df %>%
  group_by(DBYear) %>%
  arrange(DBYear, desc(sum_paper_centrality)) %>%
  slice_head(n = 3) 
# ggplot(aes(x = DBYear, y = sum_paper_centrality, group = Fullname, color = Fullname)) +
# geom_line()

# top 20 country each year, by paper's publish time
country_year_centrality_df <-
  df %>%
  mutate(PaperSecUID = as.character(PaperSecUID)) %>%
  left_join(paper_centrality_df, by = "PaperSecUID") %>%
  group_by(DBYear, Country) %>%
  summarise(sum_paper_centrality = sum(vertex_centrality)) %>%
  ungroup()

country_year_centrality_df %>%
  group_by(DBYear) %>%
  arrange(DBYear, desc(sum_paper_centrality)) %>%
  slice_head(n = 3)
# filter(Country!= "USA") %>%
# ggplot(aes(x = DBYear, y = sum_paper_centrality, group = Country, color = Country)) +
# geom_line()

`summarise()` has grouped output by 'DBYear'. You can override using the `.groups` argument.



DBYear,Fullname,sum_paper_centrality
<dbl>,<chr>,<dbl>
2015,"Greenstone, Michael",0.009213305
2015,"Card, David",0.008726687
2015,"Barreca, Alan",0.007279196
2016,"Finkelstein, Amy",0.009216040
2016,"Van Nieuwerburgh, Stijn",0.008722972
2016,"Abaluck, Jason",0.008720519
2017,"Abdulkadiroglu, Atila",0.010666142
2017,"Pathak, Parag A.",0.010666142
2017,"Angrist, Joshua D.",0.007756874


`summarise()` has grouped output by 'DBYear'. You can override using the `.groups` argument.



DBYear,Country,sum_paper_centrality
<dbl>,<chr>,<dbl>
2015,USA,0.65407502
2015,England,0.02617039
2015,Germany,0.01914502
2016,USA,0.82733627
2016,England,0.04920541
2016,Germany,0.01429075
2017,USA,0.76862900
2017,England,0.02640141
2017,Canada,0.01597741


In [14]:
# test power iteration method (similar setting to neo4j)
powerMethod(t(incidence_matrix) %*% incidence_matrix, eps = 0.0000001, maxiter = 20, plot = FALSE)


start_time <- Sys.time()
res_power <-
  powerMethod(
    t(incidence_matrix) %*% incidence_matrix,
    eps = 0.0000001,
    maxiter = 20,
    plot = FALSE
  )
end_time <- Sys.time()
end_time - start_time
# Time difference of 2.016188 mins

Warning message in rgl.init(initValue, onlyNULL):
“RGL: unable to open X11 display”
Warning message:
“'rgl.init' failed, running with 'rgl.useNULL = TRUE'.”


v1,v2,v3,v4,v5,v6,v7,v8,v9
1,0.0002389302,8.772391e-07,3.060075e-07,2.942176e-07,2.938693e-07,2.938582e-07,2.938579e-07,2.938579e-07
1,0.0007167907,1.842202e-05,6.876967e-07,1.421067e-07,1.240409e-07,1.234092e-07,1.233844e-07,1.233832e-07
1,0.0002389302,2.415916e-04,2.416730e-04,2.416764e-04,2.416767e-04,2.416767e-04,2.416767e-04,2.416767e-04
1,0.0004778604,4.834757e-04,4.836384e-04,4.836452e-04,4.836458e-04,4.836458e-04,4.836458e-04,4.836458e-04
1,0.0004778604,4.837681e-04,4.839311e-04,4.839381e-04,4.839386e-04,4.839387e-04,4.839387e-04,4.839387e-04
1,0.0004778604,4.837681e-04,4.838738e-04,4.838800e-04,4.838805e-04,4.838806e-04,4.838806e-04,4.838806e-04
1,0.0016725115,1.463586e-03,1.453991e-03,1.453598e-03,1.453583e-03,1.453582e-03,1.453582e-03,1.453582e-03
1,0.0009557209,7.267633e-04,7.257856e-04,7.257903e-04,7.257911e-04,7.257912e-04,7.257912e-04,7.257912e-04
1,0.0007167907,2.615342e-04,2.426654e-04,2.418959e-04,2.418646e-04,2.418633e-04,2.418633e-04,2.418633e-04
1,0.0004778604,2.432292e-04,2.418578e-04,2.418522e-04,2.418524e-04,2.418524e-04,2.418524e-04,2.418524e-04


Time difference of 2.007863 mins

In [15]:
# map vertex ev centrality back
vertex_centrality_pm_df <- data.frame(vertex = incidence_matrix_col_names, vertex_centrality = res_power$vector)

author_centrality_pm_df <-
  df_test %>%
  select(Fullname) %>%
  distinct(Fullname) %>%
  left_join(vertex_centrality_pm_df, by = c("Fullname" = "vertex"))

paper_centrality_pm_df <-
  df_test %>%
  select(PaperSecUID) %>%
  distinct(PaperSecUID) %>%
  mutate(PaperSecUID = as.character(PaperSecUID)) %>%
  left_join(vertex_centrality_pm_df, by = c("PaperSecUID" = "vertex"))

country_centrality_pm_df <-
  df_test %>%
  select(Country) %>%
  distinct(Country) %>%
  mutate(Country = as.character(Country)) %>%
  left_join(vertex_centrality_pm_df, by = c("Country" = "vertex"))

country_centrality_pm_df %>%
  arrange(desc(vertex_centrality)) %>%
  head(20)

sum((res_power$vector)^2)

Country,vertex_centrality
<chr>,<dbl>
USA,9.991323e-01
England,7.266104e-05
Germany,3.215409e-05
Canada,2.287496e-05
France,1.451914e-05
Spain,1.271164e-05
Italy,1.137855e-05
Switzerland,8.886806e-06
Norway,8.672131e-06


[1] 1